In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w6-p1/sample_submission.csv
/kaggle/input/2021-ai-w6-p1/pca_test.csv
/kaggle/input/2021-ai-w6-p1/pca_train.csv
/kaggle/input/2021-ai-w6-p1/train_label.csv


In [2]:
X_train = pd.read_csv("/kaggle/input/2021-ai-w6-p1/pca_train.csv")
y_train = pd.read_csv("/kaggle/input/2021-ai-w6-p1/train_label.csv")
X_test = pd.read_csv("/kaggle/input/2021-ai-w6-p1/pca_test.csv")
submit = pd.read_csv("/kaggle/input/2021-ai-w6-p1/sample_submission.csv")

print(X_train.info())
print()
print(X_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27671 entries, 0 to 27670
Columns: 256 entries, 0 to 255
dtypes: float64(256)
memory usage: 54.0 MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13629 entries, 0 to 13628
Columns: 256 entries, 0 to 255
dtypes: float64(256)
memory usage: 26.6 MB
None


In [3]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [4]:
y_train = y_train['label']
y_train #multi-class classification

0        136
1         90
2        112
3        165
4         73
        ... 
27666      9
27667    132
27668     18
27669     41
27670    114
Name: label, Length: 27671, dtype: int64

In [5]:
import torch
torch.manual_seed(1)

In [6]:
X_train_tensor = torch.FloatTensor(X_train.to_numpy())
X_test_tensor = torch.FloatTensor(X_test.to_numpy())
y_train_tensor = torch.LongTensor(y_train)

In [7]:
#y_train_tensor =  y_train_tensor.unsqueeze(dim = 1)

MLP 

layer1 = ( linear, sigmoid )  
layer2 = ( linear, sigmoid )  

BCELoss

In [8]:
print(y_train.nunique())
print(min(y_train),max(y_train))

198
0 197


In [9]:
linear1 = torch.nn.Linear(256, 198, bias = True)
linear2 = torch.nn.Linear(198, 198, bias = True)
sigmoid = torch.nn.Sigmoid()
softmax = torch.nn.Softmax()

model = torch.nn.Sequential(linear1,sigmoid, linear2)
model

Sequential(
  (0): Linear(in_features=256, out_features=198, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=198, out_features=198, bias=True)
)

In [10]:
import torch.cuda
print(torch.cuda.is_available())
torch.cuda.manual_seed(1)

True


In [11]:
model = model.cuda()
X_train_tensor = X_train_tensor.cuda()
y_train_tensor = y_train_tensor.cuda()
X_test_tensor = X_test_tensor.cuda()

In [12]:
optimizer = torch.optim.SGD(model.parameters(), lr = 1)

nb_epochs = 30000
for epoch in range(nb_epochs + 1):
    hypothesis = model(X_train_tensor)
    cost = torch.nn.functional.cross_entropy(hypothesis, y_train_tensor)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if(epoch%100==0):
        print(epoch, cost.item())

0 5.262105464935303
100 2.172497510910034
200 1.8501241207122803
300 1.6932770013809204
400 1.585477352142334
500 1.5030401945114136
600 1.4389556646347046
700 1.3879789113998413
800 1.3457356691360474
900 1.3089444637298584
1000 1.2761682271957397
1100 1.246360421180725
1200 1.219282865524292
1300 1.1916162967681885
1400 1.1683694124221802
1500 1.1420538425445557
1600 1.118309736251831
1700 1.095337152481079
1800 1.0739322900772095
1900 1.051304817199707
2000 1.0310088396072388
2100 1.0089915990829468
2200 0.9886084198951721
2300 0.9679529666900635
2400 0.9489558339118958
2500 0.9286154508590698
2600 0.9095911979675293
2700 0.8903147578239441
2800 0.870862603187561
2900 0.8515928983688354
3000 0.8337413668632507
3100 0.8141075968742371
3200 0.7960132956504822
3300 0.7778853178024292
3400 0.7609539031982422
3500 0.7424060702323914
3600 0.7261215448379517
3700 0.7078554034233093
3800 0.6919588446617126
3900 0.6742191314697266
4000 0.6588839888572693
4100 0.641558825969696
4200 0.6268366

In [13]:
with torch.no_grad():
    hypothesis = model(X_test_tensor)
    y_pred = torch.argmax(hypothesis, dim = 1)
    submit['label'] = y_pred.cpu().detach().numpy()
    
submit

,id,label
0,0,113
1,1,9
2,2,68
3,3,28
4,4,121
...,...,...
13624,13624,77
13625,13625,69
13626,13626,23
13627,13627,68


In [14]:
submit.to_csv('submission.csv', mode = 'w', index = False)